# grupo6-bot

Branch derivada da "main". É aqui onde é feita a criação do bot no Telegram.

In [ ]:
import os
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler, MessageHandler, filters

# Configurações básicas para o bot funcionar
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Classe main do BotTelegram:
class BotTelegram:

    # Definição do Token. Ele é único para cada Bot (as instruções para rodá-lo no computador de quem irá utilizar o código devem estar no README):
    def __init__(self):
        self._token = os.getenv("TOKEN_TELEGRAM")
        if not self._token:
            raise ValueError("TOKEN não encontrado.")

        self._application = ApplicationBuilder().token(self._token).build()
        self._handlers() # Já define o _handlers aqui para que mudanças neles sejam mais fáceis de serem feitas

    # Definição dos handlers (respostas principais às mensagens do usuário):
    def _handlers(self):
        self._application.add_handler(CommandHandler("start", self.start)) # Inicia o bot
        self._application.add_handler(
            MessageHandler(filters.ALL, self.tratar_mensagem) # Trata a mensagem enviada
        )

    # Mensagem inicial do bot:
    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        await update.message.reply_text(
            "Olá! Por favor, envie um áudio ou uma imagem, para que eu realize sua transcrição ou reconhecimento."
        )

    # Handler que trata a mensagem e joga para outras classes específicas para cada tarefa, a depender do que foi enviado:
    async def tratar_mensagem(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        if update.message.text:
            await self.tratar_texto(update, context)
        elif update.message.photo:
            await self.tratar_imagem(update, context)
        elif update.message.voice or update.message.audio:
            await self.tratar_audio(update, context)
        else:
            await update.message.reply_text("Desculpe, a mensagem que você enviou não é de um tipo suportado. Envie apenas imagens ou áudios!")

    # Nessa seção, define a resposta do bot para cada tipo de mensagem enviada:
    async def tratar_texto(self, update, context):
        await update.message.reply_text("Texto recebido com sucesso! No entanto, processar textos está fora do meu escopo.\nPor favor, envie somente áudios ou imagens ;)")

    async def tratar_imagem(self, update, context):
        await update.message.reply_text("Imagem recebida com sucesso! Seu processamento está em andamento...")

    async def tratar_audio(self, update, context):
        await update.message.reply_text("Áudio recebido com sucesso! Seu processamento está em andamento...")

    # Para rodar o código do bot:
    def run(self):
        self._application.run_polling()
